In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate();

 Activating environment at `C:\Users\Administrator\Documents\GitHub\16-745-hw1\Project.toml`


In [2]:
using Random, LinearAlgebra

## 一、定义问题结构和维度信息
$$ \begin{align}
    &\text{minimize}_{x} && \frac{1}{2} x^T P x + q^T x \\
    &\text{subject to} && A x = b \\
    &&& C x \leq d \\
\end{align} $$

In [3]:
struct QPData
    P::Matrix{Float64}
    q::Vector{Float64}
    A::Matrix{Float64}
    b::Vector{Float64}
    C::Matrix{Float64}
    d::Vector{Float64}
end

In [4]:
function QPData(n::Int, m::Int, p::Int)
    QPData(zeros(n,n), zeros(n), zeros(m,n), zeros(m), zeros(p,n), zeros(p))
end

QPData

In [5]:
num_eq(qp::QPData) = length(qp.b)
num_ineq(qp::QPData) = length(qp.d)

num_ineq (generic function with 1 method)

In [6]:
Base.size(qp::QPData) = (length(qp.q), num_eq(qp), num_ineq(qp))

In [7]:
objective(qp::QPData, x) = 0.5 * x'qp.P*x + qp.q'x
ceq(qp::QPData, x) = qp.A * x - qp.b
cin(qp::QPData, x) = qp.C * x - qp.d

cin (generic function with 1 method)

In [8]:
function primal_residual(qp::QPData, x, λ, μ)
    qp.P*x + qp.q + qp.A'λ + qp.C'μ
end

primal_residual (generic function with 1 method)

In [9]:
function dual_residual(qp::QPData, x, λ, μ)
    g = ceq(qp, x)
    h = cin(qp, x)
    return [g; max.(0, h)]
end

dual_residual (generic function with 1 method)

In [10]:
function complimentarity(qp::QPData, x, λ, μ)
    return [min.(0, μ); μ .* cin(qp, x)]
end

complimentarity (generic function with 1 method)

## 二、拉格朗日到增广拉格朗日

原问题可以转化为：
$$
\min_x \max_{\lambda, u \ge 0} \quad \frac{1}{2} x^T P x + q^T x + \lambda^T (Ax-b) + u^T (Cx - d) \qquad \text{问题1}
$$

仔细看内部问题
$$
\max_{\lambda, u \ge 0} \quad \frac{1}{2} x^T P x + q^T x + \lambda^T (Ax-b) + u^T (Cx - d) \qquad \text{问题2}
$$

- 如果 $Cx-d < 0$，那么$u$只能为0，所以问题2变成了
$$\max_{\lambda, u = 0} \quad \frac{1}{2} x^T P x + q^T x + \lambda^T (Ax-b) \qquad  \text{问题2.1} $$ 
- 如果 $Cx-d \ge 0$，问题2变成了
$$\max_{\lambda, u > 0} \quad \frac{1}{2} x^T P x + q^T x + \lambda^T (Ax-b) + u^T (Cx - d) \qquad  \text{问题2.2} $$

如果$(Ax-b) \neq 0$或者$(Cx - d)> 0$，那么内部问题就是无穷的，这就迫使约束满足了，所以问题1和原问题是等价的。

### 分类-问题2.2

那么如何求解问题2，由于有一个$\max$操作，非光滑的，不好直接求解。所以用近端点法，就是给一个先验值，然后使得不能离太远，这样就会形成一个关于$\lambda, u$的二阶最大化问题

$$
\max_{\lambda, u \ge 0} \quad \frac{1}{2} x^T P x + q^T x + \lambda^T (Ax-b) - \frac{1}{2 \rho} \| \lambda - \bar{\lambda} \|^2 + u^T (Cx - d) - \frac{1}{2 \rho} \| u - \bar{u} \|^2\qquad \text{问题3}
$$

问题3解的条件是：
$$
\lambda = \bar{\lambda} + \rho (Ax - b) \quad \text{条件1} \\ 
u = \max(\bar{u} + \rho (Cx -d ), 0 )  \quad \text{条件2}
$$
由于$Cx-d \ge 0, \rho>0$，如果保持$\bar{u} \ge 0$那么条件2 可以写为
$$
u = \bar{u} + \rho (Cx -d )
$$

把条件1和2带回问题3，使得问题1变为：
$$
\min_x \text{LA}_{\rho, \bar{\lambda}, \bar{u} }(x) = \quad \frac{1}{2} x^T P x + q^T x + \bar{\lambda}^T (Ax-b)+\frac{\rho}{2} \| Ax - b \|^2 + \bar{u}^T (Cx-d)+\frac{\rho}{2} \| Cx - d \|^2 
$$
梯度就是：
$$
Px + q + A^T \bar{\lambda} + \rho A^T * (Ax-b) + C^T \bar{u} + \rho C^T * (Cx-d)
$$
Hess是
$$
P + \rho A^T*A + \rho C^T*C
$$

### 分类-问题2.1

这个只要把所有关于不等式的都为0就可以了

### 更新$\lambda, u, \rho$

整体流程：

1. 求解无约束问题$\text{LA}_{\lambda, u, \rho}$
2. 按照条件2更新$\lambda, u$
3. 更新$\rho = \alpha \rho$

In [11]:
function algrad(qp::QPData, x, λ, μ,  ρ)
    # TODO: compute the gradient of the augmented Lagrangian
    # HINT: be sure to compute the active constraints!
    grad = zero(x)
    # part1 of the gradient
    grad = qp.P * x + qp.q + qp.A' * λ + ρ * qp.A' * ceq(qp, x)
    # carefully deal with inequality constraint
    numineq = num_ineq(qp::QPData)
    h = cin(qp::QPData, x)
    for i=1:numineq
        if h[i] > 0 || μ[i] > 0
            grad = grad + qp.C[i:i,:]' * μ[i] + ρ * qp.C[i:i,:]' * h[i]
        end
    end
    return grad
end

algrad (generic function with 1 method)

In [12]:
function alhess(qp::QPData, x, λ, μ,  ρ)
    # TODO: compute the Hessian of the augmented Lagrangian
    n = size(x)
#     hess = Matrix(I,n,n)
    hess = qp.P + ρ * qp.A' * qp.A
    # carefully deal with inequality constraint
    numineq = num_ineq(qp::QPData)
    h = cin(qp::QPData, x)
    for i=1:numineq
        if h[i] > 0 || μ[i] > 0
            hess = hess  + ρ * qp.C[i:i,:]' * qp.C[i:i,:]
        end
    end
    return hess
end

alhess (generic function with 1 method)

In [13]:
function dual_update(qp, x, λ, μ, ρ)
    # TODO: compute the new values for λ and μ
    λnext = copy(λ)
    μnext = copy(μ)
    g = ceq(qp, x)
    h = cin(qp, x)
    λnext = λnext + ρ*g
    μnext = μnext + ρ*h
    # Keep the dual variables for the inequality constraints in the nonnegative orthant
    for i = 1:length(μ)
        μnext[i] = max(0, μnext[i])
    end
    return λnext, μnext
end

dual_update (generic function with 1 method)

In [14]:
function newton_solve(qp::QPData, x, λ, μ, ρ; eps_inner=1e-6)
    for i = 1:10
        # Compute the gradient and the Hessian of the augmented Lagrangian
        r = algrad(qp::QPData, x, λ, μ, ρ)
        if norm(r) < eps_inner
            return x
        end
        H = alhess(qp, x, λ, μ, ρ)
        Δx = -H\r
        # TODO: Compute the Newton step
        #       A line search will help with convergence, but shouldn't be necessary for 
        #       our problem since we're providing a good guess each time
        #line search
        alpha = 1
        r = algrad(qp, x, λ, μ, ρ)
        r2 = algrad(qp, x+alpha*Δx, λ, μ, ρ)
        while norm(r2) > norm(r)
            alpha = 0.8 * alpha
            r2 = algrad(qp, x+alpha*Δx, λ, μ, ρ)
        end
        x = x + alpha*Δx 
    end
    @warn "Inner solve max iterations"
    return x
end

newton_solve (generic function with 1 method)

In [15]:
function solve_qp(qp::QPData, x0, λ0=zeros(num_eq(qp)), μ0=zeros(num_ineq(qp)); 
                    penalty_initial=10.0, 
                    penalty_scaling=10.0, 
                    eps_primal=1e-6,
                    eps_inner=1e-6,
                    max_iters=20
                )
    x = copy(x0)
    λ = copy(λ0)
    μ = copy(μ0)
    ρ = penalty_initial
    ϕ = penalty_scaling
    # Start outer loop
    for i = 1:max_iters
        # Solve the inner, unconstrained problem
        x = newton_solve(qp, x, λ, μ, ρ, eps_inner=eps_inner)
        
        # Use the new solution to update the dual variables
        λ, μ = dual_update(qp, x, λ, μ, ρ)
        
        # TODO: update the penalty parameter
        ρ = ϕ * ρ
        if norm(dual_residual(qp, x, λ, μ)) < eps_primal
            # Return the optimized variables
            return x, λ, μ
        end        
    end
    
    @warn "Outer loop max iterations"
    return x, λ, μ
end

solve_qp (generic function with 3 methods)

In [20]:
using Test, Random
using BenchmarkTools
Random.seed!(1)
# Setting up and solving a random QP
n,m,p = 10,0,15 
qp = QPData(n,m,p)
P = rand(n,n)
qp.P .= P'P   # make it P.S.D
qp.q .= randn(n)
qp.A .= randn(m,n)
qp.b .= randn(m)
qp.C .= randn(p,n)
qp.d .= randn(p)

# Initial guess
x = randn(n)

# Solve
@btime xstar, λstar, μstar = solve_qp(qp, x)
# Check optimality conditions
@test norm(primal_residual(qp, xstar, λstar, μstar)) < 1e-3
@test norm(dual_residual(qp, xstar, λstar, μstar)) < 1e-6
@test norm(complimentarity(qp, xstar, λstar, μstar)) < 1e-3

# Compare with OSQP
using OSQP, SparseArrays
model = OSQP.Model()
OSQP.setup!(model, P=sparse(qp.P), q=qp.q, A=sparse([qp.A; qp.C]), l=[qp.b; fill(-Inf,p)], u=[qp.b; qp.d],
    eps_abs=1e-6, eps_rel=1e-6,verbose=0)
@btime res = OSQP.solve!(model)
@test norm(res.x - xstar) < 1e-3
@test norm(res.y - [λstar; μstar]) < 1e-3

  103.400 μs (1630 allocations: 309.38 KiB)
  14.399 μs (10 allocations: 944 bytes)


Test Passed

In [21]:
function build_qp(q,v; mass=1, h=0.01)
    # TODO: finish the function
    
    P = diagm([mass,mass])
    qq = diagm([mass,mass])*([0;h*9.81] - v)
    A = Matrix{Float64}(undef,0,2)
    b = Vector{Float64}(undef,0)
    J = [0 1]
    C = -J .* h
    d = J*q
    # Return as a QPData type
    QPData(P,qq,A,b,C,d)
end

build_qp (generic function with 1 method)

In [22]:
function simulate_brick(q0=[0,1.], v0=[1,0.]; h=0.01, T=3.0, m=1.0)
    times = range(0, T, step=h)
    qs = [zero(q0) for t in times]
    vs = [zero(v0) for t in times]
    qs[1] .= q0
    vs[1] .= v0
    
    # TODO: Simulate the brick by solving the QP
    #  TIP: remember to update your QP after each step
    for i in 1:size(times,1)-1
        qp = build_qp(qs[i],vs[i]; mass=m,h=h)
        xstar, λstar, μstar = solve_qp(qp, vs[i])
        vs[i+1] = [xstar[1],xstar[2]]
        qs[i+1] = qs[i] + h*vs[i+1]
    end
    
    # Return the state and velocity trajectories
    return qs, vs
end

simulate_brick (generic function with 3 methods)

In [23]:
function simulate_brick_osqp(q0=[0,1.], v0=[1,0.]; h=0.01, T=3.0, m=1.0)
    times = range(0, T, step=h)
    qs = [zero(q0) for t in times]
    vs = [zero(v0) for t in times]
    qs[1] .= q0
    vs[1] .= v0
    
    # TODO: Simulate the brick by solving the QP
    #  TIP: remember to update your QP after each step
    model = OSQP.Model()
    for i in 1:size(times,1)-1
        qp = build_qp(qs[i],vs[i]; mass=m,h=h)
        # xstar, λstar, μstar = solve_qp(qp, vs[i])
        OSQP.setup!(model, P=sparse(qp.P), q=qp.q, A=sparse([qp.A; qp.C]), l=[qp.b; fill(-Inf,size(qp.d,1))], u=[qp.b; qp.d],
            eps_abs=1e-6, eps_rel=1e-6, verbose=0)
        res = OSQP.solve!(model)
        vs[i+1] = res.x
        qs[i+1] = qs[i] + h*vs[i+1]
    end
    
    # Return the state and velocity trajectories
    return qs, vs
end

simulate_brick_osqp (generic function with 3 methods)

In [25]:
using MeshCat
using GeometryTypes, Colors, CoordinateTransformations
vis = Visualizer()
setobject!(vis["brick"], HyperRectangle(Vec(0,0,0f0), 0.5*Vec(2,1,1f0)), MeshPhongMaterial(color=colorant"firebrick"))
render(vis);

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8701
└ @ MeshCat C:\Users\Administrator\.julia\packages\MeshCat\X2AUA\src\visualizer.jl:73


In [27]:
function show_sim(vis, qs, h)
    fps = Int(1/h)
    anim = MeshCat.Animation(fps)
    for (i,q) in enumerate(qs)
        atframe(anim, i) do
            settransform!(vis["brick"], Translation(q[1],0,q[2]))
        end
    end
    setanimation!(vis, anim)
end
show_sim(vis, h::Real) = show_sim(vis, simulate_brick(h=h)[1], h)
show_sim(vis, 0.01);

You can earn extra credit by making your QP solver fast. Points will be given relative to the speed of OSQP, a state-of-the-art QP solver. There will be four different levels:

1. Less than 0.5x the time of OSQP (2x slower that OSQP) (2 pts)
2. Faster than OSQP (5 pts)
3. 2x faster than OSQP (8 pts)
4. Faster than Brian's solution (about 5x faster than OSQP) (10 pts)

It will be timed on the brick simulator, see the code below for the command we will use to obtain the timing results. Tips:

- Check out the StaticArrays package
- Consider making your own solver type
- Avoid allocating new memory
- Use the BenchmarkTools package to check the performance of individual pieces
- Check out the Julia Performance Tips
- Write a version of your simulation code that uses OSQP to compare performance

In [31]:
@btime simulate_brick();

  8.525 ms (175128 allocations: 14.70 MiB)


In [32]:
@btime simulate_brick_osqp();

  4.523 ms (29117 allocations: 2.90 MiB)
